Kaggle alberga un conjunto de datos que contiene los precios de venta de viviendas para el condado de King, que incluye Seattle. Puedes descargar el conjunto de datos de Kaggle. Tu tarea es realizar bagged trees y técnicas de regresión forestal aleatoria en el conjunto de datos para predecir el precio a partir de sus características.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from sklearn.model_selection import train_test_split
# Importar árboles de ensacado y clasificadores de bosques aleatorios
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('archive (2).zip')

In [3]:
df = df.drop_duplicates()

In [4]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


## Arbol de decision

In [5]:
df.corr().sort_values(by = 'price', ascending = False).head(5)

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
price,-0.016762,1.000000,0.308350,0.525138,0.702035,0.089661,0.256794,0.266369,0.397293,0.036362,0.667434,0.605567,0.323816,0.054012,0.126434,-0.053203,0.307003,0.021626,0.585379,0.082447
sqft_living,-0.012258,0.702035,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286
grade,0.008130,0.667434,0.356967,0.664983,0.762704,0.113621,0.458183,0.082775,0.251321,-0.144674,1.000000,0.755923,0.168392,0.446963,0.014414,-0.184862,0.114084,0.198372,0.713202,0.119248
sqft_above,-0.010842,0.605567,0.477600,0.685342,0.876597,0.183512,0.523885,0.072075,0.167649,-0.158214,0.755923,1.000000,-0.051943,0.423898,0.023285,-0.261190,-0.000816,0.343803,0.731870,0.194050
sqft_living15,-0.002901,0.585379,0.391638,0.568634,0.756420,0.144608,0.279885,0.086463,0.280439,-0.092824,0.713202,0.731870,0.200355,0.326229,-0.002673,-0.279033,0.048858,0.334605,1.000000,0.183192


In [6]:
X = df[['sqft_living','grade','sqft_above','sqft_living15','bathrooms','view','sqft_basement','bedrooms','lat']]

In [7]:
y = df.price.values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [32]:
clf = RandomForestClassifier(n_estimators=20,
                             bootstrap=True,
                             oob_score=True,
                        )



In [33]:
clf.fit(X_train, y_train)



RandomForestClassifier(n_estimators=20)

In [34]:
# Predecir para una observación
clf.predict(X_test.iloc[0].values.reshape(1, -1))



array([475000.])

In [35]:
score = clf.score(X, y)
score

0.7455235275065932

In [36]:
clf.predict(X_test[0:10])



array([475000., 750000., 722000., 148000., 420000., 359000., 184500.,
       507500., 450000., 444000.])

In [37]:
# El desequilibrio de clases no es tan extremo para nuestro conjunto de datos
# Dado que se requieren etiquetas de clase para determinar el grado de desequilibrio de clases,
#  la clase imblanace se mide típicamente con respecto a la distribución del entrenamiento.
pd.DataFrame(y_train)[0].value_counts(dropna = False)



350000.0     130
450000.0     128
425000.0     118
500000.0     114
325000.0     112
            ... 
1851000.0      1
669500.0       1
541100.0       1
3168750.0      1
544900.0       1
Name: 0, Length: 3398, dtype: int64

## Preguntas para pensar y responder
Una vez que hayas completado la tarea, responde las siguientes preguntas.

¿Cuáles son las características más importantes de tu modelo?
La cantidad de arboles, si está validado con oob_score y bootstrap

¿Qué otros parámetros podrías haber intentado ajustar?

n_jobs, max_depth para ajustar la profundidad del arbol
